In [64]:
#Import necessary packages, load the point cloud

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os

pcd = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "pcd.ply"))
num = len(pcd.points)
print("Number of points before downsampling: ", num)



Number of points before downsampling:  18914399


In [65]:
#Downsample the pointcloud to make it easier to work with

pcd_downsampled = pcd.voxel_down_sample(voxel_size=0.15)
print("Number of points after downsampling: ", len(pcd_downsampled.points))
print("Reduced the amount of points by : ", (num - len(pcd_downsampled.points))/num * 100, "%")

Number of points after downsampling:  1866953
Reduced the amount of points by :  90.12946168683446 %


In [66]:
#Vizualize the downsampled pointcloud
o3d.visualization.draw_geometries([pcd_downsampled])

In [67]:
#Statistical outlier removal

pcd_filtered, ind = pcd_downsampled.remove_statistical_outlier(nb_neighbors=20, std_ratio=2)
print("Removed ", len(pcd_downsampled.points) - len(pcd_filtered.points), " points")

Removed  36856  points


In [68]:
#Vizualise the filtered pointcloud
o3d.visualization.draw_geometries([pcd_filtered])

In [136]:
#Slicing the point cloud through the tree trunks based on the z-value, between 1 and 2, 2 and 3, 3 and 4


#pcd_filtered_array = np.asarray(pcd_filtered.points)
#pcd_filtered_array = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 1.0)]
#pcd_filtered_array = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 2.0)]

#pcd_filtered_array_2 = np.asarray(pcd_filtered.points)
#pcd_filtered_array_2 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 2.0)]
#pcd_filtered_array_2 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 3.0)]

#pcd_filtered_array_3 = np.asarray(pcd_filtered.points)
#pcd_filtered_array_3 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] > 3.0)]
#pcd_filtered_array_3 = pcd_filtered_array[np.where(pcd_filtered_array[:,2] < 4.0)]


pcd_sliced_1 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 0.2), max_bound=(100, 100, 0.4)))
pcd_sliced_2 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 1.2), max_bound=(100, 100, 1.4)))
pcd_sliced_3 = pcd_filtered.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(-100, -100, 2.4), max_bound=(100, 100, 3.6)))





In [137]:
o3d.visualization.draw_geometries([pcd_sliced_1])
o3d.visualization.draw_geometries([pcd_sliced_2])
o3d.visualization.draw_geometries([pcd_sliced_3])

In [95]:
#Clustering the first slice

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd_sliced_1.cluster_dbscan(eps=0.37, min_points=5, print_progress=True)) 

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd_sliced_1.colors = o3d.utility.Vector3dVector(colors[:, :3]) #Paint point cloud with cluster colors


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 333
point cloud has 333 clusters


In [139]:
#Clustering the second slice

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels_2 = np.array(pcd_sliced_2.cluster_dbscan(eps=0.37, min_points=5, print_progress=True)) 

max_label_2 = labels_2.max()
print(f"point cloud has {max_label_2 + 1} clusters")

colors_2 = plt.get_cmap("tab20")(labels_2 / (max_label_2 if max_label_2 > 0 else 1))
colors_2[labels_2 < 0] = 0
pcd_sliced_2.colors = o3d.utility.Vector3dVector(colors_2[:, :3]) #Paint point cloud with cluster colors

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 538
point cloud has 538 clusters


In [140]:
o3d.visualization.draw_geometries([pcd_sliced_2])

In [141]:
#Putting the clusters from the first slice into individual point clouds in a list
individual_point_clouds_slice_1 = []

for cluster_label in range(max_label + 1):
    
    cluster_indices = np.where(labels == cluster_label)[0]

    cluster_points = pcd_sliced_1.select_by_index(cluster_indices)

    individual_point_clouds_slice_1.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [142]:
#Putting the clusters from the second slice into individual point clouds in a list
individual_point_clouds_slice_2 = []

for cluster_label in range(max_label_2 + 1):
    
    cluster_indices = np.where(labels == cluster_label)[0]

    cluster_points = pcd_sliced_2.select_by_index(cluster_indices)

    individual_point_clouds_slice_2.append(cluster_points)

    del(cluster_points)
    del(cluster_indices)

In [143]:
#Running ransac on each cluster in the first slice

import pyransac3d as pyrsc

circle_cloud_1 = o3d.geometry.PointCloud()

#Define the model
model_cylinder = pyrsc.Cylinder()


for pcd in individual_point_clouds_slice_1:
    
    #Loading the points as per pyransac3d documentation
    load_points = np.asarray(pcd.points) 
    
    #Fitting the circle model to the points
    center, axis, radius, inliers = model_cylinder.fit(load_points, thresh=0.05, maxIteration = 200)
    
    if 0.15 < radius < 0.4:
        
        #Selecting the inliers
        inlier_cloud = pcd.select_by_index(inliers) 
        
        #Load inlier cloud points and colors
        load_inlier_points = np.asarray(inlier_cloud.points)
        load_inlier_colors = np.asarray(inlier_cloud.colors)
        
        #Load circle_cloud points and colors
        load_circle_points = np.asarray(circle_cloud_1.points)
        load_circle_colors = np.asarray(circle_cloud_1.colors)
        
        #Make a new set of points and colors
        new_points = np.concatenate((load_inlier_points, load_circle_points), axis = 0)
        new_colors = np.concatenate((load_inlier_colors, load_circle_colors), axis = 0)
        
        #Append the new points to cloud
        circle_cloud_1.points = o3d.utility.Vector3dVector(new_points)
        circle_cloud_1.colors = o3d.utility.Vector3dVector(new_colors)
        
    else:
            continue
    
    
    
    
    

In [145]:
#Running ransac on each cluster in the second slice


circle_cloud_2 = o3d.geometry.PointCloud()

#Define the model
model_cylinder_2 = pyrsc.Cylinder()


for pcd in individual_point_clouds_slice_2:
    
    #Loading the points as per pyransac3d documentation
    load_points_2 = np.asarray(pcd.points) 
    
    if len(load_points_2) < 4:
        continue
    
    #Fitting the circle model to the points
    center, axis, radius, inliers = model_cylinder_2.fit(load_points_2, thresh=0.05, maxIteration = 200)
    
    if 0.1 < radius < 0.4:
        
        #Selecting the inliers
        inlier_cloud_2 = pcd.select_by_index(inliers) 
        
        #Load inlier cloud points and colors
        load_inlier_points_2 = np.asarray(inlier_cloud_2.points)
        load_inlier_colors_2 = np.asarray(inlier_cloud_2.colors)
        
        #Load circle_cloud points and colors
        load_circle_points_2 = np.asarray(circle_cloud_2.points)
        load_circle_colors_2 = np.asarray(circle_cloud_2.colors)
        
        #Make a new set of points and colors
        new_points_2 = np.concatenate((load_inlier_points_2, load_circle_points_2), axis = 0)
        new_colors_2 = np.concatenate((load_inlier_colors_2, load_circle_colors_2), axis = 0)
        
        #Append the new points to cloud
        circle_cloud_2.points = o3d.utility.Vector3dVector(new_points_2)
        circle_cloud_2.colors = o3d.utility.Vector3dVector(new_colors_2)
        
    else:
            continue
    
    
    
    
    

In [146]:
circle_cloud_1.paint_uniform_color([1, 0, 0])
circle_cloud_2.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([circle_cloud_2])